# 데이터 로드

In [2]:
!pip install -U sentence-transformers

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import random
import os
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2
cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
# data_path = './data/'
data_path = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/'

In [7]:
# train= pd.read_csv(f'{data_path}train.csv')
test = pd.read_csv(f'{data_path}test.csv')
train = pd.read_csv(f'{data_path}train.csv')
# train_qna = pd.read_csv(f'{data_path}train_80000_rule_based.csv')

train_ft = pd.read_csv(f'{data_path}train_ft_81900.csv')

In [5]:
category_lst = []
question_lst = []
answer_lst = []
for i in range(1, 3):
    for j in range(1, 6):
        for n in range(train.shape[0]):
            category = train['category'][n]
            question = train[f'질문_{i}'][n]
            answer = train[f'답변_{j}'][n]
            category_lst.append(category)
            question_lst.append(question)
            answer_lst.append(answer)

qna_df = pd.DataFrame({'category':category_lst,
                       'question':question_lst,
                       'answer':answer_lst})
qna_df

,category,question,answer
0,건축구조,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,건축구조,내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다."
2,건축구조,철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...
3,건축구조,철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ..."
4,건축구조,철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.
...,...,...,...
6435,타 마감하자,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...
6436,타 마감하자,벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,"AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각..."
6437,타 마감하자,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...
6438,타 마감하자,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,"창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및..."


# Similarity

In [9]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score
bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
# # bert_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
# bert_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')


def evaluate_similarity(input_text, generated_text, alpha=0.95): # cosine 가중치 조절
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

# 모델별 추론

In [58]:
kogpt_tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [18]:
kogpt_model_name = 'kogpt2_89000_0304_5epoch_0.12447loss'
kogpt_model_2 = AutoModelForCausalLM.from_pretrained(f'{data_path}model/{kogpt_model_name}').to(device)

In [19]:
kogpt_model_name = 'kogpt2_89000_0304_10epoch_0.10532loss'
kogpt_model_3 = AutoModelForCausalLM.from_pretrained(f'{data_path}model/{kogpt_model_name}').to(device)

In [20]:
kogpt_model_name = 'kogpt2_trial7_date_2_26_epoch8_loss0.1487'
kogpt_model_4 = AutoModelForCausalLM.from_pretrained(f'{data_path}models/{kogpt_model_name}').to(device)

In [21]:
kogpt_model_name = 'kogpt2_80000_5epoch_0.18720loss'
kogpt_sj_1 = AutoModelForCausalLM.from_pretrained(f'{data_path}models/{kogpt_model_name}').to(device)

In [22]:
kogpt_model_name = 'kogpt2_213486_8epoch_0.13773loss'
kogpt_sj_2 = AutoModelForCausalLM.from_pretrained(f'{data_path}models/{kogpt_model_name}').to(device)

In [23]:
kogpt_model_name = 'kogpt2_213486_10epoch_0.13645loss'
kogpt_sj_3 = AutoModelForCausalLM.from_pretrained(f'{data_path}models/{kogpt_model_name}').to(device)

In [24]:
kogpt_model_name = 'kogpt2_213486_5epoch_0.14188loss'
kogpt_sj_4 = AutoModelForCausalLM.from_pretrained(f'{data_path}models/{kogpt_model_name}').to(device)

# 1) 모델 답변 간 평가 (어떤 모델이 더 답변을 잘 내놓나?)

- 1. 정답 - 생성답변 간의 유사도
- 2. 정답 - 생성답변 간의 블루 스코어 / 글루 스코어의 평균
- 3. 모델별 생성 답변에 대한 정성 평가 (SSA)

# 2) 한 모델의 답변 간 평가 (어떤 답변이 더 좋은 답변인가?)
- 1. 질문 - 답변 간의 유사도
- 2. 질문 - 답변 간의 블루 스코어 / 글루 스코어
- 3. 생성 답변에 대한 정성 평가 (SSA)

In [59]:
def Chatbot(input_text, model, tokenizer, max_len, device, print_out=1, num_samples=5):
    model.eval()
    text = input_text.strip()
    print('Question', text)

    text = '<q>' + text + '</s><a>'
    x = tokenizer.encode(text, return_tensors='pt').to(device)
    q_len = len(text) + 1

    best_generated_text = None
    best_similarity_score = -1.0
    generated_texts = []  # 답변 후보군을 담을 리스트
    for i in range(num_samples):
        result_ids = model.generate(x,
                                    max_length=max_len,
                                    # repetition_penalty=1.0, # Rule-based 방식 사용하려면 off 해야 함
                                    num_return_sequences=1,
                                    do_sample=True,
                                    temperature=0.87,
                                    top_k=27,
                                    top_p=0.7)
        # result_ids = model.generate(x,max_length=max_len,num_beams=1,num_return_sequences=1,no_repeat_ngram_size=4,use_cache=True,do_sample=True,temperature=0.9,top_k=90,top_p=0.95,early_stopping=True)

        generated_text = tokenizer.decode(result_ids[0])
        generated_text = generated_text[q_len:-4]

        similarity_score = evaluate_similarity(text, generated_text)
        generated_texts.append((similarity_score,generated_text))
        if print_out == 1:
            print(generated_texts[i]) # 실제 돌릴땐 off

        if similarity_score > best_similarity_score:
            best_similarity_score = similarity_score
            best_generated_text = generated_text

    # print('Answer')
    return best_generated_text

# ----참고

# 모델 성능 평가
- 더 나은 문장 생성 여부를 평가할 수 있는 방법?
- Ko LLM 평가 기준  - ▲추론능력(ARC) ▲상식능력(HellaSwag) ▲언어이해력(MMLU) ▲환각방지 기능(TruthfulQA) ▲한국어상식생석능력(KoCommonGEN-V2)

#### 문제 - 답변 쌍 만들기

In [33]:
for idx, i in enumerate(test['질문'][:10]):
    print(f'질문 {idx+1}' , i)

질문 1 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
질문 2 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
질문 3 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
질문 4 철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?
질문 5 도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
질문 6 철근철골콘크리트 구조가 적용된 건물의 장단점은 무엇인가요?
질문 7 어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요. 그리고 합지벽지의 어떤 단점이 있나요?
질문 8 도배지 들뜸 현상이 발생하는 가장 일반적인 원인은 무엇인가요?
질문 9 도배지에 얼룩이 생기는 다양한 원인들에 대해서 자세히 알려주세요.
질문 10 고층 건물을 건축할 때 철골구조가 주로 선택되는 이유는 무엇인가요?


In [34]:
test_bleu_question = test['질문'].tolist()[:10]
test_bleu_question

['방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?',
 '도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?',
 '큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?',
 '철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?',
 '도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?',
 '철근철골콘크리트 구조가 적용된 건물의 장단점은 무엇인가요?',
 '어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요. 그리고 합지벽지의 어떤 단점이 있나요?',
 '도배지 들뜸 현상이 발생하는 가장 일반적인 원인은 무엇인가요?',
 '도배지에 얼룩이 생기는 다양한 원인들에 대해서 자세히 알려주세요.',
 '고층 건물을 건축할 때 철골구조가 주로 선택되는 이유는 무엇인가요?']

In [35]:
test_bleu_answer = [
    '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.',
    '도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.',
    '큐블럭은 일반 벽돌에 비해 높은 비용과 균열이 쉽게 생길 수 있는 점, 그리고 습기로 인해 하자가 발생할 수 있는 가능성이 높다는 점이 그 단점으로 꼽힙니다. 이러한 단점으로 인해, 적절한 유지보수와 관리가 필요하며, 특히 습기에 더욱 민감하므로 적절한 방법으로 관리해야 합니다. 압출법 단열판은 습기에 강하고, 곰팡이 및 세균의 증식을 방지하기 때문에 실내 환경을 개선할 수 있습니다. 또한, 열전도율이 매우 낮아 효율적인 단열 효과를 제공하며 시공이 간편하여 작업 시간을 단축할 수 있는 장점이 있습니다.',
    '철골구조는 화재 발생 시 내구성이 약해지며, 이는 건물의 붕괴로 이어질 수 있습니다. 이를 해결하기 위해 철골구조를 사용하는 고층 건물에서, 벽돌, 콘크리트 또는 석고보드와 같은 내화성 구조물을 둘러싸 화재에 의한 파손을 예방합니다.',
    '도배 후 도배지가 완전히 건조되기까지 최대 일주일이 걸릴 수도 있습니다. 도배 후 완전 건조되기 전까지 울음이 발생할 수 있습니다. 제조사 권장 건조시간(일반적으로 일주일 이내) 동안 기다려주세요. 건조기간 동안에는 온도와 습도에 유의해야 하며 일주일이 이후에도 울음이 있을 시 시공사에 연락해서 진단받아야 합니다.',
    '철근콘크리트는 기둥, 보, 내력벽, 바닥 슬라브 등의 주요 구조부가 철근 콘크리트 일체식 구조로 시공되며, 철근은 인장력과 압축력이 매우 강하고, 이 재료적 특징을 이용한 구조법이 철근콘크리트구조 입니다. 장점으로는 자유로운 형상, 일체화된 구조, 경제적인 제작, 내구성, 내화성, 내진성 등이 있으며, 단점으로는 비강도(자중에 비한 강도)가 작다는 점과 여러 가지 이유로 균열이 발생하기 쉽다는 점 등이 있습니다.',
    '개인 성향과 생활 방식에 따라 적합한 공간이 다를 수 있지만, 개별 공간은 개인의 공간과 프라이버시를 보장하는 반면에 오픈 플랜은 공간을 확장하고 연결감을 높입니다. 만약 개인적인 시간을 중요시하고 자신만의 공간에서 휴식을 취하는 것을 선호한다면 개별 공간이 더 적합할 수 있습니다. 그러나 사교적이고 가족과 함께하는 시간을 중시한다면 오픈 플랜이 보다 편리하고 적합한 선택일 수 있습니다. 합지벽지의 단점은 상대적으로 수명이 짧고 내구성이 약하여 금방 오염될 수 있으며, 오염이 발생하면 청소가 어렵고 변색이 생길 수 있다는 점입니다. 또한, 시간이 지남에 따라 모양이 훼손될 수도 있습니다.',
    '도배지 들뜸 하자는 도배지 끝 부분이 벽면에 부착되지 않는 현상입니다. 들뜸 하자는 울음 하자와 달리 도배지 끝 부분에서 발생하므로 도배지 가운데에 발생했다면 울음을 참고하셔야 합니다. 들뜸 현상이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 습도가 높으면 도배지의 접착력이 감소하여 벽면에서 떨어질 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 내외부 온도의 큰 차이 원인 : 내외부의 심한 온도 차로 결로가 생기면서 들뜸 현상이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 온도 관리에 책임이 있습니다. 해결 방법 : 냉열기를 가동하여 내외부의 온도차를 줄이고, 제습기를 가동하여 습도를 낮춰야 합니다. 3. 벽면의손상(불균형) 원인 : 벽면에 손상이 있을 경우 손상된 벽을 통해 습기가 유입하는 등의 원인으로 인해 들뜸이 발생할 가능성이 있습니다. 책임소재 : 벽면에 손상을 방생시킨 주체가 책임을 집니다. 해결 방법 : 벽면을 제대로 복원하고 평평하게 균형을 맞춘 후 재작업 해야 합니다. 해당 작업은 개인이 하기 어려우니 전문가의 도움을 받는 것을 추천합니다. ',
    '도배지의 오염(얼룩)은 국소적으로 점, 선, 면의 형태를 이루고 있습니다. 얼룩 하자가 발생하는 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습도 환경에서 도배지의 표면이 젖으면 다양한 물질이 흡수되기 쉽습니다. 이러한 물질은 표면에 얼룩 형태로 나타날 수 있습니다. 책임소재 :  건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 먼지 및 연기로 인한 오염 원인 : 도배지 표면에 먼지나 연기(담배, 조리과정에서 발생하는 연기 등)가 축적되면 얼룩이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 환경 관리의 책임이 있습니다. 해결 방법 : 필요한 경우 얼룩 제거제나 청소 용품을 사용하여 얼룩을 처리해야 합니다. 얼룩이 사라지지 않으면 부분 및 전체 도배 작업하는 것을 고려해야 합니다. 3. 화학물질에 의한 오염 원인 : 도배지가 화학물질에 노출되면 오염이 발생합니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 화학물질의 종류를 파악한 후 알맞은 방법으로 오염을 제거해야 하며 만약 오염이 제거되지 않는다면 벽지를 교체해야 합니다.',
    '철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가능하다는 것입니다. 또한, 철골구조는 화재, 지진 및 기타 자연재해로부터 건물을 보호하는 안전성이 뛰어나며, 건물 내부 공간을 유연하게 구성할 수 있는 유연성 역시 갖추고 있습니다.',
]
test_bleu_answer

['방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.',
 '도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.',
 '큐블럭은 일반 벽돌에 비해 높은 비용과 균열이 쉽게 생길 수 있는 점, 그리고 습기로 인해 하자가 발생할 수 있는 가능성이 높다는 점이 그 단점으로 꼽힙니다. 이러한 단점으로 인해, 적절한 유지보수와 관리가 필요하며, 특히 습기에 더욱 민감하므로 적절한 방법으로 관리해야 합니다. 압출법 단열판은 습기에 강하고, 곰팡이 및 세균의 증식을 방지하기 때문에 실내 환경을 개선할 수 있습니다. 또한, 열전도율이 매우 낮아 효율적인 단열 효과를 제공하며 시공이 간편하여 작업 시간을 단축할 수 있는 장점이 있습니다.',
 '철골구조는 화재 발생 시 내구성이 약해지며, 이는 건물의 붕괴로 이어질 수 있습니다. 이를 해결하기 위해 철골구조를 사용하는 고층 건물에서, 벽돌, 콘크리트 또는 석고보드와 같은 내화성 구조물을 둘

In [108]:
# qna_df['answer'].tolist()

# '''
#  질문 1. '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.',
# ',
#  질문 2.  '도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.',

#  질문 3.  '큐블럭은 일반 벽돌에 비해 높은 비용과 균열이 쉽게 생길 수 있는 점, 그리고 습기로 인해 하자가 발생할 수 있는 가능성이 높다는 점이 그 단점으로 꼽힙니다. 이러한 단점으로 인해, 적절한 유지보수와 관리가 필요하며, 특히 습기에 더욱 민감하므로 적절한 방법으로 관리해야 합니다. 압출법 단열판은 습기에 강하고, 곰팡이 및 세균의 증식을 방지하기 때문에 실내 환경을 개선할 수 있습니다. 또한, 열전도율이 매우 낮아 효율적인 단열 효과를 제공하며 시공이 간편하여 작업 시간을 단축할 수 있는 장점이 있습니다.',

#  질문 4. ' 철골구조는 화재 발생 시 내구성이 약해지며, 이는 건물의 붕괴로 이어질 수 있습니다. 이를 해결하기 위해 철골구조를 사용하는 고층 건물에서, 벽돌, 콘크리트 또는 석고보드와 같은 내화성 구조물을 둘러싸 화재에 의한 파손을 예방합니다.',

#  질문 5. ' '도배 후 도배지가 완전히 건조되기까지 최대 일주일이 걸릴 수도 있습니다. 도배 후 완전 건조되기 전까지 울음이 발생할 수 있습니다. 제조사 권장 건조시간(일반적으로 일주일 이내) 동안 기다려주세요. 건조기간 동안에는 온도와 습도에 유의해야 하며 일주일이 이후에도 울음이 있을 시 시공사에 연락해서 진단받아야 합니다.',

#  질문 6.  '철근콘크리트는 기둥, 보, 내력벽, 바닥 슬라브 등의 주요 구조부가 철근 콘크리트 일체식 구조로 시공되며, 철근은 인장력과 압축력이 매우 강하고, 이 재료적 특징을 이용한 구조법이 철근콘크리트구조 입니다. 장점으로는 자유로운 형상, 일체화된 구조, 경제적인 제작, 내구성, 내화성, 내진성 등이 있으며, 단점으로는 비강도(자중에 비한 강도)가 작다는 점과 여러 가지 이유로 균열이 발생하기 쉽다는 점 등이 있습니다.',

#  질문 7.   '개인 성향과 생활 방식에 따라 적합한 공간이 다를 수 있지만, 개별 공간은 개인의 공간과 프라이버시를 보장하는 반면에 오픈 플랜은 공간을 확장하고 연결감을 높입니다. 만약 개인적인 시간을 중요시하고 자신만의 공간에서 휴식을 취하는 것을 선호한다면 개별 공간이 더 적합할 수 있습니다. 그러나 사교적이고 가족과 함께하는 시간을 중시한다면 오픈 플랜이 보다 편리하고 적합한 선택일 수 있습니다. 합지벽지의 단점은 상대적으로 수명이 짧고 내구성이 약하여 금방 오염될 수 있으며, 오염이 발생하면 청소가 어렵고 변색이 생길 수 있다는 점입니다. 또한, 시간이 지남에 따라 모양이 훼손될 수도 있습니다.',

#  질문 8. ' 도배지 들뜸 하자는 도배지 끝 부분이 벽면에 부착되지 않는 현상입니다. 들뜸 하자는 울음 하자와 달리 도배지 끝 부분에서 발생하므로 도배지 가운데에 발생했다면 울음을 참고하셔야 합니다. 들뜸 현상이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 습도가 높으면 도배지의 접착력이 감소하여 벽면에서 떨어질 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 내외부 온도의 큰 차이 원인 : 내외부의 심한 온도 차로 결로가 생기면서 들뜸 현상이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 온도 관리에 책임이 있습니다. 해결 방법 : 냉열기를 가동하여 내외부의 온도차를 줄이고, 제습기를 가동하여 습도를 낮춰야 합니다. 3. 벽면의손상(불균형) 원인 : 벽면에 손상이 있을 경우 손상된 벽을 통해 습기가 유입하는 등의 원인으로 인해 들뜸이 발생할 가능성이 있습니다. 책임소재 : 벽면에 손상을 방생시킨 주체가 책임을 집니다. 해결 방법 : 벽면을 제대로 복원하고 평평하게 균형을 맞춘 후 재작업 해야 합니다. 해당 작업은 개인이 하기 어려우니 전문가의 도움을 받는 것을 추천합니다. ',

#  질문 9.  '도배지의 오염(얼룩)은 국소적으로 점, 선, 면의 형태를 이루고 있습니다. 얼룩 하자가 발생하는 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습도 환경에서 도배지의 표면이 젖으면 다양한 물질이 흡수되기 쉽습니다. 이러한 물질은 표면에 얼룩 형태로 나타날 수 있습니다. 책임소재 :  건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 먼지 및 연기로 인한 오염 원인 : 도배지 표면에 먼지나 연기(담배, 조리과정에서 발생하는 연기 등)가 축적되면 얼룩이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 환경 관리의 책임이 있습니다. 해결 방법 : 필요한 경우 얼룩 제거제나 청소 용품을 사용하여 얼룩을 처리해야 합니다. 얼룩이 사라지지 않으면 부분 및 전체 도배 작업하는 것을 고려해야 합니다. 3. 화학물질에 의한 오염 원인 : 도배지가 화학물질에 노출되면 오염이 발생합니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 화학물질의 종류를 파악한 후 알맞은 방법으로 오염을 제거해야 하며 만약 오염이 제거되지 않는다면 벽지를 교체해야 합니다.',

#  질문 10.  '철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가능하다는 것입니다. 또한, 철골구조는 화재, 지진 및 기타 자연재해로부터 건물을 보호하는 안전성이 뛰어나며, 건물 내부 공간을 유연하게 구성할 수 있는 유연성 역시 갖추고 있습니다.',


## 1) 유사도 기준 성능 평가

In [125]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score
# bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
# # bert_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
bert_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')


def evaluate_similarity(input_text, generated_text, alpha=0.95): # cosine 가중치 조절
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

In [126]:
def Chatbot(input_text, model, tokenizer, max_len, device, print_out=1, num_samples=5):
    model.eval()
    text = input_text.strip()
    print('Question', text)

    text = '<q>' + text + '</s><a>'
    x = tokenizer.encode(text, return_tensors='pt').to(device)
    q_len = len(text) + 1

    best_generated_text = None
    best_similarity_score = -1.0
    generated_texts = []  # 답변 후보군을 담을 리스트
    for i in range(num_samples):
        result_ids = model.generate(x,
                                    max_length=max_len,
                                    # repetition_penalty=1.0, # Rule-based 방식 사용하려면 off 해야 함
                                    num_return_sequences=1,
                                    do_sample=True,
                                    temperature=0.87,
                                    top_k=27,
                                    top_p=0.7)
        # result_ids = model.generate(x,max_length=max_len,num_beams=1,num_return_sequences=1,no_repeat_ngram_size=4,use_cache=True,do_sample=True,temperature=0.9,top_k=90,top_p=0.95,early_stopping=True)

        generated_text = tokenizer.decode(result_ids[0])
        generated_text = generated_text[q_len:-4]

        similarity_score = evaluate_similarity(text, generated_text)
        generated_texts.append((similarity_score,generated_text))
        if print_out == 1:
            print(generated_texts[i]) # 실제 돌릴땐 off

        if similarity_score > best_similarity_score:
            best_similarity_score = similarity_score
            best_generated_text = generated_text

    # print('Answer')
    return best_generated_text

- 질문 - 생성 답변간 유사도

#### A. Cosine (정답 VS Generated답변)

In [127]:
def answer_similarity_chatbot(question, answer, model, tokenizer, max_len, device, print_out=1, num_samples=5):
    generated_answer = Chatbot(question, model, tokenizer, max_len, device, print_out, num_samples)
    print('Generated Answer:', generated_answer)
    similarity_score = evaluate_similarity(answer, generated_answer,1) # 정답과 생성 답변 간 유사도 (코사인))
    return similarity_score

In [128]:
def answer_accummulate_similarity_chatbot(n, model):
    similarity_scores = []

    for i in range(n):
        print(f'>> {i+1} 번째 ')
        a = answer_similarity_chatbot(test_bleu_question[i], test_bleu_answer[i],model, kogpt_tokenizer, 200, device,0)
        print('Cosine 유사도', a)
        similarity_scores.append(a)

    print(f'>> 평균 유사도: {sum(similarity_scores) / len(similarity_scores)}')
    return sum(similarity_scores) / len(similarity_scores)

In [129]:
answer_accummulate_similarity_chatbot(10,kogpt_sj_1)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Cosine 유사도 0.9953126907348633
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상으로, 주로 다음과 같은 원인들이 있을 수 있습니다. 1. 높은 습도로 인한 금속 손상 - 원인: 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. - 책임소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. - 해결 방법: 제습기 운전 및 환기를 통해 실내 습도를 관리하고, 전문가의 도움을 받아 보수 작업을 진행해야 합니다.  2. 누수에 의한 곰팡이 발생 - 원인: 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. - 책임소재: 건물의 소유자나 거주자가 책임이 있습니다. - 해결 방법: 누수를 제거하고 곰팡이가 발생한 도배지 부분을 교체하거나 전체를 보수작업을 통해 처리합니다. 해당 작업은 전문가의 도움을 받는 것을 추천드립니다.
Cosine 유사도 0.8503577709197998
>> 3 번째 
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Generated Answer: 큐블럭의 단점은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을

0.7695499718189239

In [130]:
answer_accummulate_similarity_chatbot(10,kogpt_sj_2)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 경우 단열이 부족해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점이 단점입니다.
Cosine 유사도 0.835225522518158
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
Cosine 유사도 0.9321126937866211
>> 3 번째 
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Generated Answer: 큐블럭의 가격이 일반 벽돌에 비해 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 또한, 압출법단열판을 사용할 경우 습기에 강하고 곰팡이 

0.7097628295421601

In [131]:
answer_accummulate_similarity_chatbot(10,kogpt_sj_3)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 그리고 원목사이딩을 사용할 경우 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Cosine 유사도 0.8766922950744629
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
Cosine 유사도 0.9321126937866211
>> 3 번째 
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Generated Answer: 큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 그리고 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 

0.7572047352790833

In [132]:
answer_accummulate_similarity_chatbot(10,kogpt_sj_4)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Cosine 유사도 0.9953126907348633
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려우니 전문가의 도움을 받는 것을 추천합니다.
Cosine 유사도 0.9321126937866211
>> 3 번째 
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Generated Answer: 큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 그리고 압출법단열판은 습기에 강하고 곰팡이 및 

0.7686633408069611

In [133]:
answer_accummulate_similarity_chatbot(10,kogpt_model_2)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 또한, 원목사이딩의 장점은 있으나, 단점으로는 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축팽창이 생길 수 있는 점이 있습니다. 추가적으로 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 고려해야 합니다.
Cosine 유사도 0.4879300594329834
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 도배지에 녹이 묻은 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인  높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재  건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법  제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인  누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재  건물의 소유자나 거주자가 책임이 있습니다. 해결 방법  보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니

0.7129815578460693

In [134]:
answer_accummulate_similarity_chatbot(10,kogpt_model_3)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 또한, 원목사이딩의 장점은 있으나, 단점으로는 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축팽창이 생길 수 있는 점이 있습니다. 추가적으로 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 고려해야 합니다.
Cosine 유사도 0.4879300594329834
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 도배지에 생긴 녹색 또는 갈색의 녹은 녹오염으로 알려져 있습니다. 이런 현상이 발생하는 이유, 누구의 책임이며 해결 방법은 다음과 같습니다.1. 습도가 높은 경우에는 도배지 내부에 있는 금속이 산화되어 녹이 발생할 수 있습니다. 이 문제의 책임은 건물 소유자나 거주자에게 있습니다. 이를 해결하기 위해 제습기를 사용하거나 적절한 환기를 통해 습기를 제거하는 것이 중요합니다. 전문가의 도움을 받아 복원 작업을 권장합니다.2. 도배지가 계속 젖어 있는 경우에는 곰팡이와 녹이 생길 수 있습니다. 이 문제의 책임도 건물 소유자나 거주자에게 있습니다. 이를 해결하기 위해서는 누수를 해결하고 곰팡이와 녹이 생긴 부분 뿐만 아니라 전체 도배지를 교체하는 복원 작업이 필요합니다. 전문가의 도움을 받는 것이 좋습니다.
Cosine 유사도 0.7666419148445129
>> 3 번째 
Questi

0.7455166637897491

In [135]:
answer_accummulate_similarity_chatbot(10,kogpt_model_4)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 또한 원목사이딩의 단점은 주로 높은 비용, 유지보수의 어려움, 습기에 대한 취약성으로 인한 변형 가능성 등이 있습니다. 또한 너무 많은 습기에 노출될 경우 곰팡이와 Decayed Wood와 같은 문제가 발생할 수 있습니다.
Cosine 유사도 0.4879300594329834
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 도배지에 녹이 묻어나오는 것은 녹오염으로, 도배지 상에 녹색 또는 갈색의 녹이 발생하는 현상을 의미합니다. 이러한 현상이 발생하는 원인, 책임 소재 및 해결 방법은 다음과 같습니다.  1. 습도 문제 원인: 높은 습도로 인해 도배지 안쪽의 금속 물질이 산화되어 녹이 도배지에 발생할 수 있습니다. 책임 소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법: 제습기를 가동하거나 실내의 적정 습도를 유지하기 위해 환기를 통해 습기를 제거하는 것이 중요합니다. 전문가의 도움을 받아 보수 작업을 권장합니다.  2. 누수 원인: 누수로 도배지가 젖어 있는 상태가 계속되면 곰팡이와 녹이 발생할 수 있습니다. 책임 소재: 누수 문제는 건물의 소유자나 거주자의 관리 책임이 있습니다. 해결 방법: 누수를 해결하고 곰팡이와 녹이 발생한 부분 및 전체 도배지를 교체하는 보수 작업이 필요합니다. 이러한

0.7237784326076507

#### B. Cosine (질문 VS Generated답변)

In [136]:
def qna_similarity_Chatbot(input_text, model, tokenizer, max_len, device, print_out=1, num_samples=5):
    model.eval()
    text = input_text.strip()
    print('Question', text)

    text = '<q>' + text + '</s><a>'
    x = tokenizer.encode(text, return_tensors='pt').to(device)
    q_len = len(text) + 1

    best_generated_text = None
    best_similarity_score = -1.0
    generated_texts = []  # 답변 후보군을 담을 리스트
    for i in range(num_samples):
        result_ids = model.generate(x,
                                    max_length=max_len,
                                    # repetition_penalty=1.0, # Rule-based 방식 사용하려면 off 해야 함
                                    num_return_sequences=1,
                                    do_sample=True,
                                    temperature=0.87,
                                    top_k=27,
                                    top_p=0.7)
        # result_ids = model.generate(x,max_length=max_len,num_beams=1,num_return_sequences=1,no_repeat_ngram_size=4,use_cache=True,do_sample=True,temperature=0.9,top_k=90,top_p=0.95,early_stopping=True)

        generated_text = tokenizer.decode(result_ids[0])
        generated_text = generated_text[q_len:-4]

        similarity_score = evaluate_similarity(text, generated_text,1) # alpha 1
        generated_texts.append((similarity_score,generated_text))
        if print_out == 1:
            print(generated_texts[i]) # 실제 돌릴땐 off

        if similarity_score > best_similarity_score:
            best_similarity_score = similarity_score
            best_generated_text = generated_text

    # print('Answer')
    return best_similarity_score, best_generated_text

In [137]:
def qna_accummulate_similarity_Chatbot(n, model):
    similarity_scores = []

    for i in range(n):
        print(f'>> {i+1} 번째 ')
        a,b = qna_similarity_Chatbot(test_bleu_question[i].strip(), model, kogpt_tokenizer, 200, device,0)
        similarity_scores.append(a)
        print('Generated Answer:', b)
        print('Cosine 유사도', a)
        print(' ')


    print(f'>> 평균 유사도: {sum(similarity_scores) / len(similarity_scores)}')
    return sum(similarity_scores) / len(similarity_scores)

In [139]:
qna_accummulate_similarity_Chatbot(10,kogpt_sj_1 )

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Cosine 유사도 0.16913992166519165
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상으로, 주로 다음과 같은 원인들이 있을 수 있습니다. 1. 높은 습도로 인한 금속 손상 - 원인: 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. - 책임소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. - 해결 방법: 제습기 운전 및 환기를 통해 실내 습도를 관리하고, 전문가의 도움을 받아 보수 작업을 진행하는 것이 좋습니다.  2. 누수에 의한 곰팡이 발생 - 원인: 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. - 책임소재: 건물의 소유자나 거주자가 책임이 있습니다. - 해결 방법: 누수를 제거하고 곰팡이가 발생한 도배지 부분을 교체하거나 전체를 보수작업을 통해 처리합니다. 해당 작업은 전문가의 도움을 받는 것을 추천드립니다.
Cosine 유사도 0.4027608633041382
 
>> 3 번째 
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Generated Answer: 큐블럭의 단점은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 

0.40787304043769834

In [140]:
qna_accummulate_similarity_Chatbot(10,kogpt_sj_2 )

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 그리고 원목사이딩을 사용하는 경우 단열이 부족해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점이 단점입니다.
Cosine 유사도 0.2426387071609497
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
Cosine 유사도 0.4246826171875
 
>> 3 번째 
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Generated Answer: 큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 그리고 압출법단열판은 습기에 강하고 곰팡이 및 세균 

0.4038617730140686

In [141]:
qna_accummulate_similarity_Chatbot(10,kogpt_sj_3 )

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용할 경우 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Cosine 유사도 0.17896592617034912
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
Cosine 유사도 0.4246826171875
 
>> 3 번째 
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Generated Answer: 큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 그리고 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막

0.39137160778045654

In [142]:
qna_accummulate_similarity_Chatbot(10,kogpt_sj_4 )

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Cosine 유사도 0.16913992166519165
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다. 또한, 전문가의 도움을 받아 관련 보수작업을 
Cosine 유사도 0.4246826171875
 
>> 3 번째 
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Generated Answer: 큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단

0.42288013696670534

In [143]:
qna_accummulate_similarity_Chatbot(10,kogpt_model_2 )

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 또한, 원목사이딩의 장점은 있으나, 단점으로는 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축팽창이 생길 수 있는 점이 있습니다. 추가적으로 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 고려해야 합니다.
Cosine 유사도 0.30901485681533813
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 도배지에 녹이 묻은 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인  높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재  건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법  제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인  누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재  건물의 소유자나 거주자가 책임이 있습니다. 해결 방법  보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추

0.3814880907535553

In [144]:
qna_accummulate_similarity_Chatbot(10,kogpt_model_3 )

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 그리고 원목사이딩의 장점은 있으나, 단점으로는 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축팽창이 생길 수 있는 점이 있습니다. 추가적으로 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 고려해야 합니다.
Cosine 유사도 0.30901485681533813
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 도배지에 생긴 녹색 또는 갈색의 녹은 녹오염으로 알려져 있습니다. 이런 현상이 발생하는 이유, 누구의 책임이며 해결 방법은 다음과 같습니다.1. 습도가 높은 경우에는 도배지 내부에 있는 금속이 산화되어 녹이 발생할 수 있습니다. 이 문제의 책임은 건물 소유자나 거주자에게 있습니다. 이를 해결하기 위해 제습기를 사용하거나 적절한 환기를 통해 습기를 제거하는 것이 중요합니다. 전문가의 도움을 받아 복원 작업을 권장합니다.2. 도배지가 계속 젖어 있는 경우에는 곰팡이와 녹이 생길 수 있습니다. 이 문제의 책임도 건물 소유자나 거주자에게 있습니다. 이를 해결하기 위해서는 누수를 해결하고 곰팡이와 녹이 생긴 부분 뿐만 아니라 전체 도배지를 교체하는 복원 작업이 필요합니다. 전문가의 도움을 받는 것이 좋습니다.
Cosine 유사도 0.39270836114883423
 
>> 3 번째 


0.40022681951522826

In [145]:
qna_accummulate_similarity_Chatbot(10,kogpt_model_4 )

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 혹은, 원목사이딩의 단점은 주로 높은 비용, 유지보수의 어려움, 습기에 대한 취약성으로 인한 변형 가능성 등이 있습니다. 또한 너무 많은 습기에 노출될 경우 곰팡이와 Decayed Wood와 같은 문제가 발생할 수 있습니다.
Cosine 유사도 0.30901485681533813
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 도배지에 녹이 묻어나오는 것은 녹오염으로, 도배지 상에 녹색 또는 갈색의 녹이 발생하는 현상을 의미합니다. 이러한 현상이 발생하는 원인, 책임 소재 및 해결 방법은 다음과 같습니다.  1. 습도 문제 원인: 높은 습도로 인해 도배지 안쪽의 금속 물질이 산화되어 녹이 도배지에 발생할 수 있습니다. 책임 소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법: 제습기를 가동하거나 실내의 적정 습도를 유지하기 위해 환기를 통해 습기를 제거하는 것이 중요합니다. 전문가의 도움을 받아 보수 작업을 권장합니다.  2. 누수 원인: 누수로 도배지가 젖어 있는 상태가 계속되면 곰팡이와 녹이 발생할 수 있습니다. 책임 소재: 누수 문제는 건물의 소유자나 거주자의 관리 책임이 있습니다. 해결 방법: 누수를 해결하고 곰팡이와 녹이 발생한 부분 및 전체 도배지를 교체하는 보수 작업이 필요합니다.

0.40867801308631896

- 정답 - 생성답변 간 유사도

In [40]:
answer_accummulate_similarity_chatbot(10,kogpt_sj_3)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 그리고 원목사이딩을 사용할 경우 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Cosine 유사도 0.9413057568772085
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
Cosine 유사도 0.993579964085323
>> 3 번째 
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Generated Answer: 큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수

0.8520116455287224

In [41]:
answer_accummulate_similarity_chatbot(10,kogpt_sj_2)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 경우 단열이 부족해 충격에 약해 찍힌 자국이 잘 남는다는 단점이 있습니다
Cosine 유사도 0.8937935069987648
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
Cosine 유사도 0.993579964085323
>> 3 번째 
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Generated Answer: 큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 또한, 압출법단열판을 사용할 경우 습기에 강하고 곰팡이 및 세균 증

0.8154822681668831

In [42]:
answer_accummulate_similarity_chatbot(10,kogpt_sj_4)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Cosine 유사도 0.9969144805119587
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상으로, 주로 다음과 같은 원인들이 있을 수 있습니다. 1. 높은 습도로 인한 금속 손상 - 원인: 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. - 책임소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. - 해결방법에 대해 설명드리겠습니다. 1. 제습기 설치      - 책임소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다.        - 해결방법: 제습기를 가동하거나 환기를 통해 실내 습도를 조절하고, 전문가의 도움을 받아 보수작업을 하는 것이 좋습니다.        - 책임소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다.             - 해결방법: 제습기를 가
Cosine 유사도 0.9373684147607932
>> 3 번째 
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Generated Answer: 큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간

0.8422305177313648

In [43]:
answer_accummulate_similarity_chatbot(10,kogpt_model_4)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 혹은, 원목사이딩의 단점은 주로 높은 비용, 유지보수의 어려움, 습기에 대한 취약성으로 인한 변형 가능성 등이 있습니다. 또한 너무 많은 습기에 노출될 경우 곰팡이와 Decayed Wood와 같은 문제가 발생할 수 있습니다.
Cosine 유사도 0.797144995733749
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 도배지에 녹이 묻어나오는 것은 녹오염으로, 도배지 상에 녹색 또는 갈색의 녹이 발생하는 현상을 의미합니다. 이러한 현상이 발생하는 원인, 책임 소재 및 해결 방법은 다음과 같습니다.  1. 습도 문제 원인: 높은 습도로 인해 도배지 안쪽의 금속 물질이 산화되어 녹이 도배지에 발생할 수 있습니다. 책임 소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법: 제습기를 가동하거나 실내의 적정 습도를 유지하기 위해 환기를 통해 습기를 제거하는 것이 중요합니다. 전문가의 도움을 받아 보수 작업을 권장합니다.  2. 누수 원인: 누수로 도배지가 젖어 있는 상태가 계속되면 곰팡이와 녹이 발생할 수 있습니다. 책임 소재: 누수 문제는 건물의 소유자나 거주자의 관리 책임이 있습니다. 해결 방법: 누수를 해결하고 곰팡이와 녹이 발생한 부분 및 전체 도배지를 교체하는 보수 작업이 필요합니다. 이러한

0.8528998530659256

In [44]:
answer_accummulate_similarity_chatbot(10,kogpt_model_3)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 그리고 원목사이딩의 장점은 있으나, 단점으로는 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축팽창이 생길 수 있는 점이 있습니다. 추가적으로 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 고려해야 합니다.
Cosine 유사도 0.8099446783705455
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 도배지에 생긴 녹색 또는 갈색의 녹은 녹오염으로 알려져 있습니다. 이런 현상이 발생하는 이유, 누구의 책임이며 해결 방법은 다음과 같습니다.1. 습도가 높은 경우에는 도배지 내부에 있는 금속이 산화되어 녹이 발생할 수 있습니다. 이 문제의 책임은 건물 소유자나 거주자에게 있습니다. 이를 해결하기 위해 제습기를 사용하거나 적절한 환기를 통해 습기를 제거하는 것이 중요합니다. 전문가의 도움을 받아 복원 작업을 권장합니다.2. 도배지가 계속 젖어 있는 경우에는 곰팡이와 녹이 생길 수 있습니다. 이 문제의 책임도 건물 소유자나 거주자에게 있습니다. 이를 해결하기 위해서는 누수를 해결하고 곰팡이와 녹이 생긴 부분 뿐만 아니라 전체 도배지를 교체하는 복원 작업이 필요합니다. 전문가의 도움을 받는 것이 좋습니다.
Cosine 유사도 0.9005433638890583
>> 3 번째 
Questi

0.8034126823462617

In [45]:
answer_accummulate_similarity_chatbot(10,kogpt_model_2)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated Answer: 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 그리고 원목사이딩의 장점은 있으나, 단점으로는 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축팽창이 생길 수 있는 점이 있습니다. 추가적으로 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 고려해야 합니다.
Cosine 유사도 0.8099446783705455
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated Answer: 도배지에 녹이 묻은 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 이 현상은 일반적으로 고습하거나 누수가 있는 경우에 발생할 수 있습니다. 습도가 높거나 누수가 있는 경우에는 도배지 주변의 온도가 부족하거나 도배지 내부의 금속이 부식되어 녹이 발생할 수 있습니다. 따라서 도배지에 녹오염이 발생하는 경우에는 건물 소유자나 거주자에게 도움을 요청하여 문제를 해결하는 것이 중요합니다.
Cosine 유사도 0.8684491311084657
>> 3 번째 
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Generated Answer: 큐블럭의 주요 단점으로는 일반 벽돌에 비해 비싼 가격과 균열 발생 가능성, 습기로 인한 하자 발생 가능성이 있습니다. 또한, 설치 시 추가적으로 접착재와 전문 기술이 요구되어 비용이 더

0.7989903297274878

In [8]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score
bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
# # bert_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
# bert_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')


def evaluate_similarity(input_text, generated_text, alpha=0.95): # cosine 가중치 조절
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [77]:
def Chatbot(input_text, model, tokenizer, max_len, device, print_out=1, num_samples=5):
    model.eval()
    text = input_text.strip()
    print('Question', text)

    text = '<q>' + text + '</s><a>'
    x = tokenizer.encode(text, return_tensors='pt').to(device)
    q_len = len(text) + 1

    best_generated_text = None
    best_similarity_score = -1.0
    generated_texts = []  # 답변 후보군을 담을 리스트
    for i in range(num_samples):
        result_ids = model.generate(x,
                                    max_length=max_len,
                                    # repetition_penalty=1.0, # Rule-based 방식 사용하려면 off 해야 함
                                    num_return_sequences=1,
                                    do_sample=True,
                                    temperature=0.87,
                                    top_k=27,
                                    top_p=0.7)
        # result_ids = model.generate(x,max_length=max_len,num_beams=1,num_return_sequences=1,no_repeat_ngram_size=4,use_cache=True,do_sample=True,temperature=0.9,top_k=90,top_p=0.95,early_stopping=True)

        generated_text = tokenizer.decode(result_ids[0])
        generated_text = generated_text[q_len:-4]

        similarity_score = evaluate_similarity(text, generated_text)
        generated_texts.append((similarity_score,generated_text))
        if print_out == 1:
            print(generated_texts[i]) # 실제 돌릴땐 off

        if similarity_score > best_similarity_score:
            best_similarity_score = similarity_score
            best_generated_text = generated_text

    # print('Answer')
    return best_generated_text

In [18]:
print(Chatbot(test['질문'].tolist()[2].strip(), kogpt_sj_1, kogpt_tokenizer, 200, device,0))


Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Answer
큐블럭의 단점은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.


In [95]:
reset_seeds(SEED)

max_len = 200 ## len 200일때가 더 나음 ?

print('kogpt_sj_1')
print(Chatbot(test['질문'].tolist()[2].strip(), kogpt_sj_1, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_sj_2')
print(Chatbot(test['질문'].tolist()[2].strip(), kogpt_sj_2, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_sj_3')
print(Chatbot(test['질문'].tolist()[2].strip(), kogpt_sj_3, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_sj_4')
print(Chatbot(test['질문'].tolist()[2].strip(), kogpt_sj_4, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_model_2')
print(Chatbot(test['질문'].tolist()[2].strip(), kogpt_model_2, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_model_3')
print(Chatbot(test['질문'].tolist()[2].strip(), kogpt_model_3, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_model_4')
print(Chatbot(test['질문'].tolist()[2].strip(), kogpt_model_4, kogpt_tokenizer, max_len, device))
print(' ')

# 점수는 유사도(jacc0.05), 코사인 유사도

kogpt_sj_1
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
(0.6427792611329453, '큐블럭의 단점은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.')
(0.6427792611329453, '큐블럭의 단점은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.')
(0.6427792611329453, '큐블럭의 단점은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.')
(0.6427792611329453, '큐블럭의 단점은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.')
(0.6427792611329453, '큐블럭의 단점은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.')
큐블럭의 단점은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점입

In [96]:
reset_seeds(SEED)

max_len = 200 ## len 200일때가 더 나음 ?

print('kogpt_sj_1')
print(Chatbot(test['질문'].tolist()[13].strip(), kogpt_sj_1, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_sj_2')
print(Chatbot(test['질문'].tolist()[13].strip(), kogpt_sj_2, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_sj_3')
print(Chatbot(test['질문'].tolist()[13].strip(), kogpt_sj_3, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_sj_4')
print(Chatbot(test['질문'].tolist()[13].strip(), kogpt_sj_4, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_model_2')
print(Chatbot(test['질문'].tolist()[13].strip(), kogpt_model_2, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_model_3')
print(Chatbot(test['질문'].tolist()[13].strip(), kogpt_model_3, kogpt_tokenizer, max_len, device))
print(' ')

print('kogpt_model_4')
print(Chatbot(test['질문'].tolist()[13].strip(), kogpt_model_4, kogpt_tokenizer, max_len, device))
print(' ')

# 점수는 유사도(jacc0.05), 코사인 유사도

kogpt_sj_1
Question 방염벽지가 실크벽지와 합지벽지와 다른 점은 무엇인가요?
(0.5308547735214233, '실크벽지는 종이벽지에 PVC 코팅이 된 형태의 벽지입니다. 이러한 벽지는 내구성이 강하고 변색이 적으며, 보통 1평당 약 6,000원 정도의 가격이 필요합니다. 그러나 시공이 상대적으로 어렵고, 화재 시 유해가스를 발생시킬 수 있는 단점도 있습니다.')
(0.647209334373474, '실크벽지는 합지벽지와는 달리 PVC 코팅을 하고 방염처리가 된 제품입니다. 이로 인해 실크벽지는 매우 부식 및 변형이 가능하며, 내구성이 다소 떨어질 수 있다는 단점이 있습니다. 그러므로 실크벽지 위에 벽지를 도배할 때에는 신중한 판단이 필요합니다.')
(0.5942574679851532, '실크벽지는 합지벽지와는 달리 PVC로 코팅을 하고 방염처리가 된 제품입니다. 이로 인해 실크벽지는 보온성이 부족하고, 온도 변화에 민감하여 변형이 발생할 수 있습니다. 따라서 실크벽지는 비닐로 한 번 코팅된 재질이며, 바로 위에 붙이지 않고 사용하는 것이 좋습니다. 다만, 이는 단점입니다.')
(0.6057622462511062, '방염벽지는 PVC 코팅을 하고 방염처리가 된 벽지로, 화재 시 유독가스의 확산속도를 지연시키는 장점이 있습니다. 이러한 특성으로 화재로부터의 안전을 증대시킬 수 있지만, 시공이 어렵고 비용이 상대적으로 높다는 단점이 있습니다. 일반적으로 실크벽지는 미술과 같은 구조물에서 사용되며, 일반적으로 실크벽지보다 안전한 환경을 선호합니다.')
(0.5308547735214233, '실크벽지는 종이벽지에 PVC 코팅이 된 형태의 벽지입니다. 이러한 벽지는 내구성이 강하고 변색이 적으며, 보통 1평당 약 6,000원 정도의 가격이 필요합니다. 그러나 시공이 상대적으로 어렵고, 화재 시 유해가스를 발생시킬 수 있는 단점도 있습니다.')
실크벽지는 합지벽지와는 달리 PVC 코팅을 하고 방염처리가 된 제품입니다. 이로 인해 실크벽지는 매우 부식 및

## 2) BLEU 스코어  
- 테스트 셋에 대한 예상답변 10개를 만들고, 유사도 및 블루 스코어를 확인하기

#### A.BLEU (정답 VS Generated답변)

In [ ]:
# !pip install nltk

In [76]:
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu_score(question, answer, model, tokenizer, max_len, print_out, num_samples=5):
    max_len = max_len
    device = 'cuda' if cuda else 'cpu'

    generated_answer = Chatbot(question, model, tokenizer, max_len, device, print_out, num_samples=5)
    print('Answer :', answer)
    print('Generated :', generated_answer)
    # print('BLEU Score :', corpus_bleu([[answer]], [generated_answer]))
    # print(' ')
    return 'BLEU Score', corpus_bleu([[answer]], [generated_answer])

In [35]:
test_bleu_question[0]

'방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?'

In [75]:
def accumulate_bleu_score(n, model):
    bleu_scores = []
    for i in range(n):
        print(f'>> {i+1} 번째 ')
        a,b = calculate_bleu_score(test_bleu_question[i], test_bleu_answer[i],model, kogpt_tokenizer, max_len, 0)
        # print(test_bleu_question[i])
        # print(test_bleu_answer[i])
        print(f'>>   BLEU Score: {b}')
        print(' ')
        bleu_scores.append(b)

    print(f'>> 평균 BLEU Score: {sum(bleu_scores) / len(bleu_scores)}')
    return sum(bleu_scores) / len(bleu_scores)

In [40]:
model = kogpt_model_4
tokenizer = kogpt_tokenizer
max_len = 200

reset_seeds(SEED)

calculate_bleu_score(test_bleu_question[0], test_bleu_answer[0],model, tokenizer, max_len, 0)

Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Generated : 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 혹은, 원목사이딩의 단점은 주로 높은 비용, 유지보수의 어려움, 습기에 대한 취약성으로 인한 변형 가능성 등이 있습니다. 또한 너무 많은 습기에 노출될 경우 곰팡이와 Decayed Wood와 같은 문제가 발생할 수 있습니다.
 


('BLEU Score', 0.2418198945831229)

In [42]:
reset_seeds(SEED)

max_len = 200 ## len 200일때가 더 나음 ?

print('kogpt_sj_1')
print(calculate_bleu_score(test_bleu_question[0], test_bleu_answer[0],kogpt_sj_1, tokenizer, max_len, 0))
print(' ')

print('kogpt_sj_2')
print(calculate_bleu_score(test_bleu_question[0], test_bleu_answer[0],kogpt_sj_2, tokenizer, max_len, 0))
print(' ')

print('kogpt_sj_3')
print(calculate_bleu_score(test_bleu_question[0], test_bleu_answer[0],kogpt_sj_3, tokenizer, max_len, 0))
print(' ')

print('kogpt_sj_4')
print(calculate_bleu_score(test_bleu_question[0], test_bleu_answer[0],kogpt_sj_4, tokenizer, max_len, 0))
print(' ')

print('kogpt_model_2')
print(calculate_bleu_score(test_bleu_question[0], test_bleu_answer[0],kogpt_model_2, tokenizer, max_len, 0))
print(' ')

print('kogpt_model_3')
print(calculate_bleu_score(test_bleu_question[0], test_bleu_answer[0],kogpt_model_3, tokenizer, max_len, 0))
print(' ')

print('kogpt_model_4')
print(calculate_bleu_score(test_bleu_question[0], test_bleu_answer[0],kogpt_model_4, tokenizer, max_len, 0))
print(' ')


kogpt_sj_1
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
('BLEU Score', 0.9672922423621466)
 
kogpt_sj_2
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 경우 단열이 부족해 충격에 약해 찍힌 자국이 잘 남는다는 단점이 있습니다
('BLEU Score', 0.6434658855124847)
 
kogpt_sj_3
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이

In [55]:
accumulate_bleu_score(10,kogpt_sj_1) # 0.395064820528519

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
>>   BLEU Score: 0.9672922423621466
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Answer : 도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
Generated : 도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어

0.395064820528519

In [59]:
accumulate_bleu_score(10,kogpt_sj_2)  #0.19850016161793593

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 경우 단열이 부족해 충격에 약해 찍힌 자국이 잘 남는다는 단점이 있습니다
>>   BLEU Score: 0.6434658855124847
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Answer : 도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
Generated : 도배지에 녹이 묻어나오는 것은 녹오염으로, 도배지 상에 녹색 또는 갈색의 녹이 발생하는 현상을 의

0.19850016161793593

In [97]:
accumulate_bleu_score(10,kogpt_sj_3)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용할 경우 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
>>   BLEU Score: 0.8335556053135148
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Answer : 도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
Generated : 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는

0.3875909025515456

In [98]:
accumulate_bleu_score(10,kogpt_sj_4)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
>>   BLEU Score: 0.9672922423621466
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Answer : 도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
Generated : 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상으로, 주

0.30117306299429936

In [61]:
accumulate_bleu_score(10,kogpt_model_2)  # 0.3300929995696934

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Generated : 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 그리고 원목사이딩의 장점은 있으나, 단점으로는 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축팽창이 생길 수 있는 점이 있습니다. 추가적으로 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 고려해야 합니다.
>>   BLEU Score: 0.3217258555300093
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Answer : 도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰

0.3300929995696934

In [62]:
accumulate_bleu_score(10,kogpt_model_3)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Generated : 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 그리고 원목사이딩의 장점은 있으나, 단점으로는 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축팽창이 생길 수 있는 점이 있습니다. 추가적으로 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 고려해야 합니다.
>>   BLEU Score: 0.3217258555300093
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Answer : 도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰

0.333583206097929

In [79]:
accumulate_bleu_score(10,kogpt_model_4)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Generated : 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 혹은, 원목사이딩의 단점은 주로 높은 비용, 유지보수의 어려움, 습기에 대한 취약성으로 인한 변형 가능성 등이 있습니다. 또한 너무 많은 습기에 노출될 경우 곰팡이와 Decayed Wood와 같은 문제가 발생할 수 있습니다.
>>   BLEU Score: 0.2418198945831229
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Answer : 도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수

0.3503061128983157

#### B. BLEU (질문 VS Generated답변)

In [66]:
from nltk.translate.bleu_score import corpus_bleu

def qna_calculate_bleu_score(question, model, tokenizer, max_len, print_out, num_samples=5):
    max_len = max_len
    device = 'cuda' if cuda else 'cpu'

    generated_answer = Chatbot(question, model, tokenizer, max_len, device, print_out, num_samples=5)
    print('Question :', question)
    print('Generated :', generated_answer)
    # print('BLEU Score :', corpus_bleu([[answer]], [generated_answer]))
    # print(' ')
    return 'BLEU Score', corpus_bleu([[question]], [generated_answer])

In [71]:
def qna_accumulate_bleu_score(n, model):
    bleu_scores = []
    for i in range(n):
        print(f'>> {i+1} 번째 ')
        a,b = qna_calculate_bleu_score(test_bleu_question[i], model, tokenizer, max_len, 0)
        # print(test_bleu_question[i])
        # print(test_bleu_answer[i])
        print(f'>>   BLEU Score: {b}')
        print(' ')
        bleu_scores.append(b)

    print(f'>> 평균 BLEU Score: {sum(bleu_scores) / len(bleu_scores)}')
    return sum(bleu_scores) / len(bleu_scores)


In [70]:
reset_seeds(SEED)
tokenizer = kogpt_tokenizer
max_len = 200

print('kogpt_sj_1')
print(qna_calculate_bleu_score(test_bleu_question[0], kogpt_sj_1, tokenizer, max_len, 0))
print(' ')

print('kogpt_sj_2')
print(qna_calculate_bleu_score(test_bleu_question[0], kogpt_sj_2, tokenizer, max_len, 0))
print(' ')

print('kogpt_sj_3')
print(qna_calculate_bleu_score(test_bleu_question[0], kogpt_sj_3, tokenizer, max_len, 0))
print(' ')

print('kogpt_sj_4')
print(qna_calculate_bleu_score(test_bleu_question[0], kogpt_sj_4, tokenizer, max_len, 0))
print(' ')

print('kogpt_model_2')
print(qna_calculate_bleu_score(test_bleu_question[0], kogpt_model_2, tokenizer, max_len, 0))
print(' ')

print('kogpt_model_3')
print(qna_calculate_bleu_score(test_bleu_question[0], kogpt_model_3, tokenizer, max_len, 0))
print(' ')

print('kogpt_model_4')
print(qna_calculate_bleu_score(test_bleu_question[0], kogpt_model_4, tokenizer, max_len, 0))
print(' ')


kogpt_sj_1
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
('BLEU Score', 0.1413312056926513)
 
kogpt_sj_2
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 경우 단열이 부족해 충격에 약해 찍힌 자국이 잘 남는다는 단점이 있습니다
('BLEU Score', 0.22491349899472515)
 
kogpt_sj_3
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용할 경우 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 

In [72]:
qna_accumulate_bleu_score(10, kogpt_sj_1)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
>>   BLEU Score: 0.1413312056926513
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Question : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated : 도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 보통 높은 습도나 누수로 인해 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지 안쪽의 금속이 녹을 베어나올 수 있습니다. 따라서 공간 내 습도를 적정 수준으로 유지하고, 도배지 주변의 누수를 조사하고 보수하는 것이 중요합니다. 또한, 도배지 주변의 누수를 조사하고 보수하는 것이 중요합니다.  만일 누수로 인해 도배지가 젖어서 곰팡이가 발생하게 된다면, 녹오염 뿐만 아니라 곰팡이 번식할 수 있으므로 누수 부분을 보수하고, 곰팡이가 번식한 도배지 일부 또는 전체를 교체해야 합니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다. 따라서 도배지에 녹이 묻어나오는 경우에는 공간 내 습도 
>>   BLEU Score: 0.02445778438718587
 
>> 3 번째 
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인

0.08188328442585421

In [99]:
qna_accumulate_bleu_score(10, kogpt_sj_2)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 그리고 원목사이딩을 사용하는 경우 단열이 부족해 충격에 약해 찍힌 자국이 잘 남는다는 단점이 있습니다
>>   BLEU Score: 0.19957366191407863
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Question : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated : 도배지에 녹이 묻어나오는 것은 녹오염으로, 도배지 상에 녹색 또는 갈색의 녹이 발생하는 현상을 의미합니다. 이러한 경우에는 물을 천으로 가볍게 닦아주어 해결 방법으로는 될 수 있습니다. 하지만 만약 누수가 심각한 경우, 전문가의 도움을 받아야 합니다.
>>   BLEU Score: 0.11179072766045815
 
>> 3 번째 
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Question : 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Generated : 큐블럭의 가격이 일반 벽돌에 비해 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 또한, 압출법단열판을 사용할 때는 습기조절, 단열 및 해충 유입 방지 효과가 있고 흙이 주 재료이기에 매우 친환경적인 점이 장점입니다.
>>   BLEU Score: 0.14795273594591055
 
>> 4 번째 
Question 철골구조를 사용하는 고층 건물에서,

0.09366941698642431

In [100]:
qna_accumulate_bleu_score(10, kogpt_sj_3)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 그리고 원목사이딩을 사용할 경우 단열이 부족하고 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
>>   BLEU Score: 0.14450256172527007
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Question : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated : 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
>>   BLEU Score: 0.03896596332297933
 
>> 3 번째 
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열

0.06989888855313546

In [101]:
qna_accumulate_bleu_score(10, kogpt_sj_4)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
>>   BLEU Score: 0.1413312056926513
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Question : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated : 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다. 그리고 해당 작업은 개인이 하기 어려운 
>>   BLEU Score: 0.0377855590538856
 
>> 3 번째 
Question 

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Question : 도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
Generated : 도배 후 도배지가 완전히 건조되기까지 최대 일주일이 걸릴 수도 있습니다.
>>   BLEU Score: 3.320819814171721e-78
 
>> 6 번째 
Question 철근철골콘크리트 구조가 적용된 건물의 장단점은 무엇인가요?


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Question : 철근철골콘크리트 구조가 적용된 건물의 장단점은 무엇인가요?
Generated : 철근철골콘크리트 구조는 콘크리트의 압축 강도를 활용하면서도 철근의 인장 강도를 활용하여, 철근의 인장 강도를 콘크리트의 간극 또는 공극에 설치하여 철근의 인장 강도를 보완하는 구조입니다. 이 구조는 철골의 강도와 철근의 인장 강도를 결합하여 건축물의 안정성을 확보하는 데에 도움을 줄 수 있습니다. 철근의 강도는 콘크리트의 인장 응력에 대한 고려가 필요하며, 철근의 내장을 통해 콘크리트의 약점을 보완하여 안정한 건축물을 건설하는 것이 중요합니다.
>>   BLEU Score: 0.05152705468200393
 
>> 7 번째 
Question 어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요. 그리고 합지벽지의 어떤 단점이 있나요?
Question : 어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요. 그리고 합지벽지의 어떤 단점이 있나요?
Generated : 개별 공간은 개인 공간과 더 많은 프라이버시를 제공하며, 오픈 플랜은 공간을 확장하고 연결감을 높입니다. 그리고 합지벽지의 단점은 상대적으로 수명이 짧고 내구성이 약하며 오염이 생기면 닦거나 청소가 어렵고 시간이 지나면 변색이 될 수 있는 점이 단점입니다.
>>   BLEU Score: 0.1928470158294425
 
>> 8 번째 
Question 도배지 들뜸 현상이 발생하는 가장 일반적인 원인은 무엇인가요?
Question : 도배지 들뜸 현상이 발생하는 가장 일반적인 원인은 무엇인가요?
Generated : 도배지 들뜸 현상이 발생하는 가장 일반적인 원인은 공간 내 습도가 높거나 내외부 온도의 큰 차이, 그리고 벽면의 불균형이 해당됩니다. 습도가 높으면 도배지의 접착력이 감소하여 벽면에서 떨어질 수 있고, 내외부의 심한 온도 차로 결로가 생기면서 들뜸 현상이 발생할 수 있습니다. 또한 벽면에 손상이 있을 경

0.07028005268823657

In [102]:
qna_accumulate_bleu_score(10, kogpt_model_2)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated : 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 그리고 원목사이딩의 장점은 있으나, 단점으로는 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축팽창이 생길 수 있는 점이 있습니다. 추가적으로 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 고려해야 합니다.
>>   BLEU Score: 0.06907520970044465
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Question : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated : 도배지에 녹이 묻은 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인  높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재  건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법  제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인  누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재  

0.0604557812949309

In [103]:
qna_accumulate_bleu_score(10, kogpt_model_3)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated : 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 그리고 원목사이딩의 장점은 있으나, 단점으로는 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축팽창이 생길 수 있는 점이 있습니다. 추가적으로 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 고려해야 합니다.
>>   BLEU Score: 0.06907520970044465
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Question : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated : 도배지에 생긴 녹색 또는 갈색의 녹은 녹오염으로 알려져 있습니다. 이런 현상이 발생하는 이유, 누구의 책임이며 해결 방법은 다음과 같습니다.1. 습도가 높은 경우에는 도배지 내부에 있는 금속이 산화되어 녹이 발생할 수 있습니다. 이 문제의 책임은 건물 소유자나 거주자에게 있습니다. 이를 해결하기 위해 제습기를 사용하거나 적절한 환기를 통해 습기를 제거하는 것이 중요합니다. 전문가의 도움을 받아 복원 작업을 권장합니다.2. 도배지가 계속 젖어 있는 경우에는 곰팡이와 녹이 생길 수 있습니다. 이 문제의 책임도 건물 소유자나 거주자에게 있습니다. 이를 해결하기 위해서는

0.05915893870874665

In [104]:
qna_accumulate_bleu_score(10, kogpt_model_4)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated : 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 혹은, 원목사이딩의 단점은 주로 높은 비용, 유지보수의 어려움, 습기에 대한 취약성으로 인한 변형 가능성 등이 있습니다. 또한 너무 많은 습기에 노출될 경우 곰팡이와 Decayed Wood와 같은 문제가 발생할 수 있습니다.
>>   BLEU Score: 0.07123850690617634
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Question : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated : 도배지에 녹이 묻어나오는 것은 녹오염으로, 도배지 상에 녹색 또는 갈색의 녹이 발생하는 현상을 의미합니다. 이러한 현상이 발생하는 원인, 책임 소재 및 해결 방법은 다음과 같습니다.  1. 습도 문제 원인: 높은 습도로 인해 도배지 안쪽의 금속 물질이 산화되어 녹이 도배지에 발생할 수 있습니다. 책임 소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법: 제습기를 가동하거나 실내의 적정 습도를 유지하기 위해 환기를 통해 습기를 제거하는 것이 중요합니다. 전문가의 도움을 받아 보수 작업을 권장합니다.  2. 누수 원인: 누수로 도배지가 젖어 있는 상태가 계속되면

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Question : 철근철골콘크리트 구조가 적용된 건물의 장단점은 무엇인가요?
Generated : 철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. 철골은 강하고 가벼운 소재로, 높은 내구성과 강도를 가지고 있어 대규모 공간이나 강성이 요구되는 대형 고층건물에 적합합니다. 그러나 화재에 취약하고 녹이 슬면 강성이 떨어지는 단점이 있습니다. 이를 보완하기 위해 철근과 콘크리트를 섞어 사용하는 방식이 개발되었습니다. 철근콘크리트는 철근이 인장력을, 콘크리트가 압축력을 담당하여 강성을 가지며 콘크리트의 열전도가 낮은 재료상 특징을 이용해 서로의 단점을 보완하도록 결합한 구조입니다. 내화성, 내구성, 내진성능이 뛰어나지만 자중이 크고 시공과정이 복잡하며 공사기간이 길고 균일한 시공이 불가능하다는 단점이 있습니다. 철근철골콘크리트는 이 두 가지 구조의 장점을 살려 만든 것으로, 철골 뼈대로 구조를 세우고 그 주변을 철근과 콘크리트로 감싸거나, 철근과 콘크리트로 구조를 만들고 그 위에 철골을 올리는 형태로
>>   BLEU Score: 0.0250731198977466
 
>> 7 번째 
Question 어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요. 그리고 합지벽지의 어떤 단점이 있나요?
Question : 어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요. 그리고 합지벽지의 어떤 단점이 있나요?
Generated : 개인 성향과 생활 방식에 따라 적합한 공간이 다를 수 있지만, 개별 공간은 개인의 공간과 프라이버시를 보장하는 반면에 오픈 플랜은 공간을 확장하고 연결감을 높입니다. 만약 개인적인 시간을 중요시하고 자신만의 공간에서 휴식을 취하는 것을 선호한다면 개별 공간이 더 적합할 수 있습니다. 그러나 사교적이고 가족과 함께하는 시간을 중시한다면 오픈 플랜이 보다 편리하고 적합한 선택일 수 있습니다. 한편, 합지벽지의 단점은 상대적으로 수명이 짧

0.05444937370767996

## 3) GLEU 스코어
- BLEU 보다 문장길이에 덜 영향받는 평가 방식

In [70]:
def bleu_score(reference, candidate):
    bleu = corpus_bleu([[reference]], [candidate])
    return bleu

In [71]:
import nltk

def gleu_score(reference, candidate):
    reference = [reference.split()]
    candidate = candidate.split()
    gleu = nltk.translate.gleu_score.sentence_gleu(reference, candidate)
    return gleu

# 예제 사용법
reference_sentence = "This is a reference sentence."
candidate_sentence = "This is a candidate sentence."

score1 = gleu_score(reference_sentence, candidate_sentence)
score2 = bleu_score(reference_sentence, candidate_sentence)
print(f"GLEU Score: {score1}")
print(f"BLEU Score: {score2}")


GLEU Score: 0.5
BLEU Score: 0.6649394793371532


In [81]:
('BLEU Score', 0.2418198945831229)

('BLEU Score', 0.2418198945831229)

#### A. GLEU (정답 VS Generated답변)

In [66]:
!pip install nltk

In [67]:

def calculate_gleu_score(question, answer, model, tokenizer, max_len, print_out, num_samples=5):
    max_len = max_len
    device = 'cuda' if cuda else 'cpu'

    generated_answer = Chatbot(question, model, tokenizer, max_len, device, print_out, num_samples=5)
    print('Answer :', answer)
    print('Generated :', generated_answer)
    # print('BLEU Score :', corpus_bleu([[answer]], [generated_answer]))
    # print(' ')

    reference = [answer.split()]
    candidate = generated_answer.split()
    gleu = nltk.translate.gleu_score.sentence_gleu(reference, candidate)

    return 'GLEU Score', gleu

In [68]:

def accumulate_gleu_score(n, model):
    gleu_scores = []
    for i in range(n):
        print(f'>> {i+1} 번째 ')
        a,b = calculate_gleu_score(test_bleu_question[i], test_bleu_answer[i],model, kogpt_tokenizer, 200, 0)
        # print(test_bleu_question[i])
        # print(test_bleu_answer[i])
        print(f'>>   GLEU Score: {b}')
        print(' ')
        gleu_scores.append(b)

    print(f'>> 평균 GLEU Score: {sum(gleu_scores) / len(gleu_scores)}')
    return sum(gleu_scores) / len(gleu_scores)

In [72]:
accumulate_gleu_score(10, kogpt_sj_1)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
>>   GLEU Score: 0.8979591836734694
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Answer : 도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
Generated : 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상으로, 주

0.3125854636009362

In [73]:
accumulate_gleu_score(10, kogpt_sj_2)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 경우 단열이 부족해 충격에 약해 찍힌 자국이 잘 남는다는 단점이 있습니다
>>   GLEU Score: 0.3723404255319149
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Answer : 도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
Generated : 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는

0.1820565611758498

In [74]:
accumulate_gleu_score(10, kogpt_sj_3)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 그리고 원목사이딩을 사용할 경우 단열이 부족하거나 조도가 달라질 수 있습니다.
>>   GLEU Score: 0.3723404255319149
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Answer : 도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
Generated : 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 

0.27016938024896536

In [75]:
accumulate_gleu_score(10, kogpt_sj_4)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
>>   GLEU Score: 0.8979591836734694
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Answer : 도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
Generated : 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상으로, 주

0.26559471156433767

In [76]:
accumulate_gleu_score(10, kogpt_model_2)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Generated : 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 그리고 원목사이딩의 장점은 있으나, 단점으로는 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축팽창이 생길 수 있는 점이 있습니다. 추가적으로 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 고려해야 합니다.
>>   GLEU Score: 0.09872611464968153
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Answer : 도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 

0.19313787102519916

In [77]:
accumulate_gleu_score(10, kogpt_model_3)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Generated : 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 그리고 원목사이딩의 장점은 있으나, 단점으로는 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축팽창이 생길 수 있는 점이 있습니다. 추가적으로 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 고려해야 합니다.
>>   GLEU Score: 0.09872611464968153
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Answer : 도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 

0.11060915045815747

In [78]:
accumulate_gleu_score(10, kogpt_model_4)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Answer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
Generated : 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 혹은, 원목사이딩의 단점은 주로 높은 비용, 유지보수의 어려움, 습기에 대한 취약성으로 인한 변형 가능성 등이 있습니다. 또한 너무 많은 습기에 노출될 경우 곰팡이와 Decayed Wood와 같은 문제가 발생할 수 있습니다.
>>   GLEU Score: 0.01935483870967742
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Answer : 도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 

0.1476589869816996

#### B. GLEU (질문 VS Generated답변)

In [84]:
import nltk

def qna_calculate_gleu_score(question, model, tokenizer, max_len, print_out, num_samples=5):
    max_len = max_len
    device = 'cuda' if cuda else 'cpu'

    generated_answer = Chatbot(question, model, tokenizer, max_len, device, print_out, num_samples=5)
    print('Question :', question)
    print('Generated :', generated_answer)
    # print('BLEU Score :', corpus_bleu([[answer]], [generated_answer]))
    # print(' ')

    reference = [question.split()]
    candidate = generated_answer.split()
    gleu = nltk.translate.gleu_score.sentence_gleu(reference, candidate)

    return 'GLEU Score', gleu

In [85]:
def qna_accumulate_gleu_score(n, model):
    gleu_scores = []
    for i in range(n):
        print(f'>> {i+1} 번째 ')
        a,b = qna_calculate_gleu_score(test_bleu_question[i], model, kogpt_tokenizer, 200, 0)
        # print(test_bleu_question[i])
        # print(test_bleu_answer[i])
        print(f'>>   GLEU Score: {b}')
        print(' ')
        gleu_scores.append(b)

    print(f'>> 평균 GLEU Score: {sum(gleu_scores) / len(gleu_scores)}')
    return sum(gleu_scores) / len(gleu_scores)


In [86]:
qna_accumulate_gleu_score(10, kogpt_sj_1)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
>>   GLEU Score: 0.01020408163265306
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Question : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated : 도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 생기는 것을 말합니다. 이 현상은 일반적으로 높은 습도나 누수로 인해 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹오염이 발생할 수 있습니다. 따라서 공간 내 습도를 적정하게 유지하기 위해 제습기를 가동하고, 환기를 통해 습도를 조절하는 것이 필요합니다. 또한, 도배지 주변의 누수를 조사하고 보수하는 것이 중요합니다.  만일 누수로 인해 도배지가 젖어서 곰팡이가 발생하게 된다면, 녹오염 뿐만 아니라 곰팡이까지 발생할 수 있습니다. 따라서 누수 부분을 보수하고, 곰팡이가 번식한 도배지 일부 또는 전체를 교체해야 합니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다. 따라서 도배지에 녹이 발생하는 경우, 공간 내 습도 및 누수 원인을 파악하여 필요한 경우 보수 작업을 
>>   GLEU Score: 0.004608294930875576
 
>> 3 번째 
Question 큐블럭의 단점을 알

0.013417572184589693

In [87]:
qna_accumulate_gleu_score(10, kogpt_sj_2)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 경우 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점이 단점입니다.
>>   GLEU Score: 0.05660377358490566
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Question : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated : 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
>>   GLEU Score: 0.00684931506849315
 
>> 3 번째 
Question 큐블럭의 단점을 알려주세

0.018307814638638585

In [88]:
qna_accumulate_gleu_score(10, kogpt_sj_3)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 그리고 원목사이딩을 사용할 경우 단열이 부족하고 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
>>   GLEU Score: 0.010638297872340425
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Question : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated : 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
>>   GLEU Score: 0.00684931506849315
 
>> 3 번째 
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단

0.010852239139681918

In [89]:
qna_accumulate_gleu_score(10, kogpt_sj_4)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
>>   GLEU Score: 0.01020408163265306
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Question : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated : 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상으로, 주로 다음과 같은 원인들이 있을 수 있습니다. 1. 높은 습도로 인한 금속 손상 - 원인: 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. - 책임소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. - 해결방법에 대해 설명드리겠습니다. 1. 제습기 가동 - 원인: 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법: 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 - 원인: 누수로 도배지가 젖어 있는 상태가 계속되면 곰팡이가 발생할 수 있습니다. 책임소재: 건물의 소유자나 거주자가 책임이 있습니다
>>   GLEU Score: 0.004694835680751174
 
>> 3 번째 
Question 큐블럭의 단점을 알려주세요. 또한, 압출법 단열

0.012863282202913977

In [90]:
qna_accumulate_gleu_score(10, kogpt_model_2)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated : 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 또한, 원목사이딩의 장점은 있으나, 단점으로는 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축팽창이 생길 수 있는 점이 있습니다. 추가적으로 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 고려해야 합니다.
>>   GLEU Score: 0.012738853503184714
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Question : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated : 도배지에 녹이 묻은 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 이 현상은 일반적으로 높은 습도나 누수로 인해 발생할 수 있습니다.  고습으로 인해 도배지 안쪽의 금속이 녹아서 녹이 도배지에 드러나게 됩니다. 이를 해결하기 위해서는 습도를 조절하고, 제습기를 사용하여 적정 습도를 유지하는 것이 중요합니다. 또한, 도배지 내부의 금속이 녹아서 녹을 도배지에 드러나게 되므로, 적절한 방법으로 녹을 제거하거나, 도배지를 복원하고 균형을 맞춘 후 재작업하는 것이 좋습니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다. 따라서 도배지에 녹이 묻

0.016926619603753516

In [91]:
qna_accumulate_gleu_score(10, kogpt_model_3)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated : 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 그리고 원목사이딩의 장점은 있으나, 단점으로는 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축팽창이 생길 수 있는 점이 있습니다. 추가적으로 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 고려해야 합니다.
>>   GLEU Score: 0.009554140127388535
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Question : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated : 도배지에 생긴 녹색 또는 갈색의 녹은 녹오염으로 알려져 있습니다. 이런 현상이 발생하는 이유, 누구의 책임이며 해결 방법은 다음과 같습니다.1. 습도가 높은 경우에는 도배지 내부에 있는 금속이 산화되어 녹이 발생할 수 있습니다. 이 문제의 책임은 건물 소유자나 거주자에게 있습니다. 이를 해결하기 위해 제습기를 사용하거나 적절한 환기를 통해 습기를 제거하는 것이 중요합니다. 전문가의 도움을 받아 복원 작업을 권장합니다.2. 도배지가 계속 젖어 있는 경우에는 곰팡이와 녹이 생길 수 있습니다. 이 문제의 책임도 건물 소유자나 거주자에게 있습니다. 이를 해결하기 위해서

0.019795326743470537

In [92]:
qna_accumulate_gleu_score(10, kogpt_model_4)

>> 1 번째 
Question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Generated : 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다. 혹은, 원목사이딩의 단점은 주로 높은 비용, 유지보수의 어려움, 습기에 대한 취약성으로 인한 변형 가능성 등이 있습니다. 또한 너무 많은 습기에 노출될 경우 곰팡이와 Decayed Wood와 같은 문제가 발생할 수 있습니다.
>>   GLEU Score: 0.00967741935483871
 
>> 2 번째 
Question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Question : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Generated : 도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 보통 높은 습도나 누수로 인해 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹오염이 발생할 수 있습니다. 따라서 공간 내 습도를 적정하게 유지하기 위해 제습기를 가동하고, 환기를 통해 습도를 조절하는 것이 필요합니다. 또한, 도배지 주변의 누수를 조사하고 보수하는 것이 중요합니다.  만일 누수로 인해 도배지가 젖어서 곰팡이가 발생하게 된다면, 녹오염 뿐만 아니라 곰팡이까지 발생할 수 있습니다. 따라서 이때는 누수 부

0.012644682520083972

## 4)  정답 VS 질문

In [93]:
bleu_scores = []
gleu_scores = []

for i in range(10) :
    print(f'{i+1} 번째 ')

    glue = gleu_score(test_bleu_question[i],test_bleu_answer[i])
    blue = bleu_score(test_bleu_question[i],test_bleu_answer[i])

    bleu_scores.append(blue)
    gleu_scores.append(glue)

    print(f' >> GLEU Score: {glue}')
    print(f' >> BLEU Score: {blue}')

print(' ')
print('평균 GLEU Score', sum(gleu_scores) / len(gleu_scores) *100)
print('평균 BLEU Score', sum(bleu_scores) / len(bleu_scores) *10)

1 번째 
 >> GLEU Score: 0.0
 >> BLEU Score: 0.11449338673409044
2 번째 
 >> GLEU Score: 0.00904977375565611
 >> BLEU Score: 0.045830334437250674
3 번째 
 >> GLEU Score: 0.0070921985815602835
 >> BLEU Score: 0.06105562396127922
4 번째 
 >> GLEU Score: 0.0847457627118644
 >> BLEU Score: 0.1867242752616464
5 번째 
 >> GLEU Score: 0.006329113924050633
 >> BLEU Score: 0.06131467831922992
6 번째 
 >> GLEU Score: 0.0
 >> BLEU Score: 0.031284977792542704
7 번째 
 >> GLEU Score: 0.02830188679245283
 >> BLEU Score: 0.10512370968591227
8 번째 
 >> GLEU Score: 0.012012012012012012
 >> BLEU Score: 0.026906494340243524
9 번째 
 >> GLEU Score: 0.003115264797507788
 >> BLEU Score: 0.021724415382760823
10 번째 
 >> GLEU Score: 0.014492753623188406
 >> BLEU Score: 0.08266855169748455
 
평균 GLEU Score 1.6513876619829246
평균 BLEU Score 0.7371264476124405


In [94]:
bleu_scores = []
gleu_scores = []

for i in range(10-1) :
    print(f'{i+1} 번째 ')

    glue = gleu_score(test_bleu_question[i],test_bleu_answer[i+1])
    blue = bleu_score(test_bleu_question[i],test_bleu_answer[i+1])

    bleu_scores.append(blue)
    gleu_scores.append(glue)

    print(f' >> GLEU Score: {glue}')
    print(f' >> BLEU Score: {blue}')

print(' ')
print('평균 GLEU Score', sum(gleu_scores) / len(gleu_scores) *100)
print('평균 BLEU Score', sum(bleu_scores) / len(bleu_scores) *10)

1 번째 
 >> GLEU Score: 0.0
 >> BLEU Score: 0.0174263019014115
2 번째 
 >> GLEU Score: 0.0035460992907801418
 >> BLEU Score: 1.0061732372666604e-78
3 번째 
 >> GLEU Score: 0.00847457627118644
 >> BLEU Score: 0.0583861185732668
4 번째 
 >> GLEU Score: 0.0
 >> BLEU Score: 0.02917801890136067
5 번째 
 >> GLEU Score: 0.0
 >> BLEU Score: 2.8558168296729336e-155
6 번째 
 >> GLEU Score: 0.0
 >> BLEU Score: 0.011435063036624216
7 번째 
 >> GLEU Score: 0.0
 >> BLEU Score: 6.65162424532223e-79
8 번째 
 >> GLEU Score: 0.003115264797507788
 >> BLEU Score: 0.0165266959269436
9 번째 
 >> GLEU Score: 0.0
 >> BLEU Score: 3.5419162525628293e-155
 
평균 GLEU Score 0.1681771151052708
평균 BLEU Score 0.1477246648217853


#### A. BLEU (질문 VS 정답)

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

def correct_answer_calculate_bleu_score(question, answer):
    print('Question :', question)
    print('Anaswer :', answer)
    # print('BLEU Score :', corpus_bleu([[answer]], [generated_answer]))
    # print(' ')
    return 'BLEU Score', corpus_bleu([[question]], [answer])

In [ ]:
def correct_answer_accumulate_bleu_score(n, model):
    bleu_scores = []
    for i in range(n):
        print(f'>> {i+1} 번째 ')
        a,b = correct_answer_calculate_bleu_score(test_bleu_question[i], test_bleu_answer[i])
        # print(test_bleu_question[i])
        # print(test_bleu_answer[i])
        print(f'>>   BLEU Score: {b}')
        print(' ')
        bleu_scores.append(b)

    print(f'>> 평균 BLEU Score: {sum(bleu_scores) / len(bleu_scores)}')
    return sum(bleu_scores) / len(bleu_scores)

In [ ]:
correct_answer_accumulate_bleu_score(10, kogpt_sj_1)

>> 1 번째 
Question : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Anaswer : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
>>   BLEU Score: 0.11449338673409044
 
>> 2 번째 
Question : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
Anaswer : 도배지 녹은 자국은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.
>>   BLEU Score: 0.045830334437250674
 
>> 3 번째 
Question : 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
Anaswer : 큐블럭은 일반 벽돌에 비해 높은 비용과 균열이 쉽게 생길 수 있는 점, 그리고 습기로 인해 하자가 발생할 수 있는 가능성이 높다는 점이 그 단점으로 꼽힙니다

0.07371264476124405

In [ ]:
from bleu.bleu import Bleu
from meteor.meteor import Meteor
from rouge.rouge import Rouge
from cider.cider import Cider
from spice.spice import Spice
import spacy
import argparse
import codecs

def tokenize_sentences(sentences):
    nlp = spacy.load("en_core_web_sm")
    nlp.pipeline = [('tagger', nlp.tagger)]

    tokenized_sentences = []
    for sentence in sentences:
        tokens = ' '.join(token.text for token in nlp(sentence))
        tokenized_sentences.append(tokens.rstrip())

    return tokenized_sentences

def tokenize(dict):
    for key in dict:
        dict[key] = tokenize_sentences(dict[key])

    return dict

def evaluate_similarity(input_text, generated_text):
    # Your similarity calculation logic here
    pass

def evaluator(gts, res):
    print('Tokenization...')
    gts = tokenize(gts)
    res = tokenize(res)

    print('Setting up scorers...')
    scorers = [
        (Bleu(4), ["Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"]),
        (Meteor(), "METEOR"),
        # (Rouge(), "ROUGE_L"),
        (Cider(), "CIDEr"),
        (Spice(), "SPICE")
    ]

    for scorer, method in scorers:
        print(f'Computing {scorer.method()} score...')
        score, scores = scorer.compute_score(gts, res)

        if isinstance(method, list):
            for sc, scs, m in zip(score, scores, method):
                print(f'{m}: {sc:.3f}')
        else:
            print(f'{method}: {score:.3f}')

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--key_file', default="", type=str)
    parser.add_argument('--gts_file', default="", type=str)
    parser.add_argument('--res_file', default="", type=str)
    args = parser.parse_args()

    key_file = args.key_file
    gts_file = args.gts_file
    res_file = args.res_file

    gts = {}
    res = {}

    with codecs.open(key_file, encoding='utf-8') as f:
        key_lines = f.readlines()
    with codecs.open(gts_file, encoding='utf-8') as f:
        gts_lines = f.readlines()
    with codecs.open(res_file, encoding='utf-8') as f:
        res_lines = f.readlines()

    for key_line, gts_line, res_line in zip(key_lines, gts_lines, res_lines):
        key = '#'.join(key_line.rstrip('\n').split(' '))
        if key not in gts:
            gts[key] = [gts_line.rstrip('\n')]
            res[key] = [res_line.rstrip('\n')]
        else:
            gts[key].append(gts_line.rstrip('\n'))
            res[key].append(res_line.rstrip('\n'))

    evaluator(gts,
